### Import libraries


In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pathlib import Path, PurePath
import pandas as pd
import requests
from requests.exceptions import HTTPError, ConnectionError
from ipywidgets import interact
import ipywidgets as widgets
import re
from ipywidgets import interact
import ipywidgets as widgets
import pandas as pd
from IPython.display import display

!pip install nltk
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 

### Connect to personal google drive google drive to enable data download
Requires you to have this data available on your personal drive: https://drive.google.com/open?id=1ZVxvPnrnA8ffGoFsVxJs75QL9li6AfG7

In [0]:
from google.colab import drive # for connecting to dataset on personal google drive
# mount personal google drive that has data uploaded (Requires verification)
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Download data

In [0]:
# upload data and list contents
input_dir = PurePath('/content/drive/My Drive/CORD-19-research-challenge')
list(Path(input_dir).glob('*'))

[PosixPath('/content/drive/My Drive/CORD-19-research-challenge/biorxiv_medrxiv'),
 PosixPath('/content/drive/My Drive/CORD-19-research-challenge/comm_use_subset'),
 PosixPath('/content/drive/My Drive/CORD-19-research-challenge/noncomm_use_subset'),
 PosixPath('/content/drive/My Drive/CORD-19-research-challenge/custom_license'),
 PosixPath('/content/drive/My Drive/CORD-19-research-challenge/metadata.readme'),
 PosixPath('/content/drive/My Drive/CORD-19-research-challenge/json_schema.txt'),
 PosixPath('/content/drive/My Drive/CORD-19-research-challenge/COVID.DATA.LIC.AGMT.pdf'),
 PosixPath('/content/drive/My Drive/CORD-19-research-challenge/metadata.csv')]

In [0]:
metadata_path = input_dir / 'metadata.csv'
metadata = pd.read_csv(metadata_path,
                               dtype={'Microsoft Academic Paper ID': str,
                                      'pubmed_id': str})

# Set the abstract to the paper title if it is null
metadata.abstract = metadata.abstract.fillna(metadata.title)
print("Number of articles before removing duplicates: %s " % len(metadata))

Number of articles before removing duplicates: 44220 


In [0]:
# Some papers are duplicated since they were collected from separate sources. Thanks Joerg Rings
duplicate_paper = ~(metadata.title.isnull() | metadata.abstract.isnull() | metadata.publish_time.isnull()) & (metadata.duplicated(subset=['title', 'abstract']))
metadata.dropna(subset=['publish_time', 'journal'])
metadata = metadata[~duplicate_paper].reset_index(drop=True)
print("Number of articles AFTER removing duplicates: %s " % len(metadata))

Number of articles AFTER removing duplicates: 42713 


### **TODO**

In [0]:
# REMOVE articles missing publish_date or journal name
print("Number of articles AFTER removing missing date and journal: %s " % len(metadata))

Number of articles AFTER removing missing date and journal: 42713 


### Create Data Classes for the Research Dataset and Papers
These classes make it easier to navigate through the datasources. There is a class called ResearchPapers that wraps the entire dataset an provide useful functions to navigate through it, and Paper, that make it easier to view each paper.

In [0]:
def get(url, timeout=6):
    try:
        r = requests.get(url, timeout=timeout)
        return r.text
    except ConnectionError:
        print(f'Cannot connect to {url}')
        print(f'Remember to turn Internet ON in the Kaggle notebook settings')
    except HTTPError:
        print('Got http error', r.status, r.text)

# Convert the doi to a url
def doi_url(d): 
    return f'http://{d}' if d.startswith('doi.org') else f'http://doi.org/{d}'


class ResearchPapers:
    
    def __init__(self, metadata: pd.DataFrame):
        self.metadata = metadata
        
    def __getitem__(self, item):
        return Paper(self.metadata.iloc[item])
    
    def __len__(self):
        return len(self.metadata)
    
    def head(self, n):
        return ResearchPapers(self.metadata.head(n).copy().reset_index(drop=True))
    
    def tail(self, n):
        return ResearchPapers(self.metadata.tail(n).copy().reset_index(drop=True))
    
    def abstracts(self):
        return self.metadata.abstract.dropna()
    
    def titles(self):
        return self.metadata.title.dropna()
        
    def _repr_html_(self):
        return self.metadata._repr_html_()
    
class Paper:
    
    '''
    A single research paper
    '''
    def __init__(self, item):
        self.paper = item.to_frame().fillna('')
        self.paper.columns = ['Value']
    
    def doi(self):
        return self.paper.loc['doi'].values[0]
    
    def html(self):
        '''
        Load the paper from doi.org and display as HTML. Requires internet to be ON
        '''
        if self.doi():
            url = doi_url(self.doi()) 
            text = get(url)
            return widgets.HTML(text)
    
    def text(self):
        '''
        Load the paper from doi.org and display as text. Requires Internet to be ON
        '''
        text = get(self.doi())
        return text
    
    def abstract(self):
        return self.paper.loc['abstract'].values[0]
    
    def title(self):
        return self.paper.loc['title'].values[0]
    
    def authors(self, split=False):
        '''
        Get a list of authors
        '''
        authors = self.paper.loc['authors'].values[0]
        if not authors:
            return []
        if not split:
            return authors
        if authors.startswith('['):
            authors = authors.lstrip('[').rstrip(']')
            return [a.strip().replace("\'", "") for a in authors.split("\',")]
        
        # Todo: Handle cases where author names are separated by ","
        return [a.strip() for a in authors.split(';')]
        
    def _repr_html_(self):
        return self.paper._repr_html_()
    

papers = ResearchPapers(metadata)

#### Show a Paper

In [0]:
papers[1]

,Value
sha,
source_x,Elsevier
title,Coronaviruses in Balkan nephritis
doi,10.1016/0002-8703(80)90355-5
pmcid,
pubmed_id,6243850
license,els-covid
abstract,Coronaviruses in Balkan nephritis
publish_time,1980-03-31
authors,"Georgescu, Leonida; Diosi, Peter; Buţiu, Ioan;..."


#### Pull info from a paper

In [0]:
index=1
paper=papers[index]
print("Example paper #%s\nTitle: %s\nAuthors: %s " % (index, paper.title(), paper.authors(split=True)))


Example paper #1
Title: Coronaviruses in Balkan nephritis
Authors: ['Georgescu, Leonida', 'Diosi, Peter', 'Buţiu, Ioan', 'Plavoşin, Livia', 'Herzog, Georgeta'] 


### Text Preprocessing
To prepare the text for the search index we perform the following steps
1.   Remove punctuations and special characters
2.   Convert to lowercase
3.   Tokenize into individual tokens (words mostly)
4.   Remove stopwords like (and, to))
5.   Lemmatize

In [0]:
# Download the stop words we plan on using
nltk.download("punkt")
nltk.download("stopwords")
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [0]:
# Hardcode the data we want to use in search
SEARCH_DISPLAY_COLUMNS = ['title', 'abstract', 'doi', 'authors', 'journal', 'publish_time']

In [0]:
english_stopwords = list(set(stopwords.words('english')))

def strip_characters(text):
    t = re.sub('\(|\)|:|,|;|\.|’|”|“|\?|%|>|<', '', text)
    t = re.sub('/', ' ', t)
    t = t.replace("'",'')
    return t

def clean(text):
    t = text.lower()
    t = strip_characters(t)
    return t

def tokenize(text):
    words = nltk.word_tokenize(text)
    return list(set([word for word in words 
                     if len(word) > 1
                     and not word in english_stopwords
                     and not (word.isnumeric() and len(word) is not 4)
                     and (not word.isnumeric() or word.isalpha())] )
               )
    
def lemmatize(word_list,lemmatizer):
    # Init the Wordnet Lemmatizer
    lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in word_list])
    return lemmatized_output


def preprocess(text):
    t = clean(text)
    tokens = tokenize(t)
    lemmatizer=WordNetLemmatizer()
    tokens = lemmatize(tokens,lemmatizer)
    return tokens

In [0]:
class SearchResults:
    
    def __init__(self, 
                 data: pd.DataFrame,
                 columns = None):
        self.results = data
        if columns:
            self.results = self.results[columns]
            
    def __getitem__(self, item):
        return Paper(self.results.loc[item])
    
    def __len__(self):
        return len(self.results)
        
    def _repr_html_(self):
        return self.results._repr_html_()

class WordTokenIndex:
    
    def __init__(self, 
                 corpus: pd.DataFrame, 
                 columns=SEARCH_DISPLAY_COLUMNS):
        self.corpus = corpus
        raw_search_str = self.corpus.abstract.fillna('') + ' ' + self.corpus.title.fillna('')
        self.index = raw_search_str.apply(preprocess).to_frame()
        self.index.columns = ['terms']
        self.index.index = self.corpus.index
        self.columns = columns

    
    def search(self, search_string):
        search_terms = preprocess(search_string)
        result_index = self.index.terms.apply(lambda terms: any(i in terms for i in search_terms))
        results = self.corpus[result_index].copy().reset_index().rename(columns={'index':'paper'})
        return SearchResults(results, self.columns + ['paper'])

### Creating a search index¶ - Using a RankBM25 Search Index
We will create a simple search index that will just match search tokens in a document. First we tokenize the abstract and store it in a dataframe. Then we just match search terms against it.

RankBM25 is a python library that implements algorithms for a simple search index. https://pypi.org/project/rank-bm25/

In [0]:
!pip install rank_bm25
# Create a prebaked search engine with existing package: https://pypi.org/project/rank-bm25/
from rank_bm25 import BM25Okapi

In [0]:
class RankBM25Index(WordTokenIndex):
    
    def __init__(self, corpus: pd.DataFrame, columns=SEARCH_DISPLAY_COLUMNS):
        super().__init__(corpus, columns)
        self.bm25 = BM25Okapi(self.index.terms.tolist())
        
    def search(self, search_string, n=4):
        search_terms = preprocess(search_string)
        doc_scores = self.bm25.get_scores(search_terms)
        ind = np.argsort(doc_scores)[::-1][:n]
        results = self.corpus.iloc[ind][self.columns]
        results['Score'] = doc_scores[ind]
        results = results[results.Score > 0]
        return SearchResults(results.reset_index(), self.columns + ['Score'])

### Create the index (This takes several minutes)

In [0]:
bm25_index = RankBM25Index(metadata)

### Search by date

In [0]:
# example output
query='curise ship'
n=50
results = bm25_index.search(query,n)
results.results.sort_values(by=['publish_time'], ascending=False).head(5)

,title,abstract,doi,authors,journal,publish_time,Score
23,Inhibition of SARS-CoV-2 infection (previously...,AbstractThe recent outbreak of coronavirus dis...,10.1101/2020.03.09.983247,Shuai Xia; Meiqin Liu; Chao Wang; Wei Xu; Qiao...,NaN,2020-03-12,59.883227
44,Will novel virus go pandemic or be contained?,The repatriation of 565 Japanese citizens from...,10.1126/science.367.6478.610,"Kupferschmidt, Kai; Cohen, Jon",Science,2020-02-06,59.863036
33,Qu’apprend-t-on de nouveau des épidémies émerg...,Points essentiels L’Afrique et l’Asie du Sud-E...,10.1016/j.lpm.2019.09.036,"Malvy, Denis; Gaüzère, Bernard-Alex; Migliani,...",La Presse Médicale,2019-12-31,59.869735
10,Alpha herpesvirus egress and spread from neuro...,Alpha herpesviruses naturally infect the perip...,10.1101/729830,"Ambrosini, A. E.; Deshmukh, N.; Berry, M. J.; ...",NaN,2019-08-08,59.938204
31,A Comprehensive Review of Autophagy and Its Va...,Autophagy (self-eating) is a conserved cellula...,10.3390/cells8070674,"Khandia, Rekha; Dadar, Maryam; Munjal, Ashok; ...",Cells,2019 Jul 3,59.871299


In [0]:
# example output
query='ACE spike'
n=50
results = bm25_index.search(query,n)
results.results.sort_values(by=['publish_time'], ascending=False).head(5)

,title,abstract,doi,authors,journal,publish_time,Score
39,Will novel virus go pandemic or be contained?,The repatriation of 565 Japanese citizens from...,10.1126/science.367.6478.610,"Kupferschmidt, Kai; Cohen, Jon",Science,2020-02-06,48.658186
6,EGR1 upregulation following Venezuelan equine ...,Abstract Venezuelan equine encephalitis virus ...,10.1016/j.virol.2019.10.016,"Dahal, Bibha; Lin, Shih-Chao; Carey, Brian D.;...",Virology,2020-01-02,48.803227
24,Identification of a Novel Linear B-Cell Epitop...,"Porcine deltacoronavirus (PDCoV), first identi...",10.3390/ijms21020648,"Fu, Jiayu; Chen, Rui; Hu, Jingfei; Qu, Huan; Z...",Int J Mol Sci,2020 Jan 19,48.691775
10,Clinician perceptions of respiratory infection...,Abstract Outbreaks of emerging and re-emerging...,10.1016/j.idh.2019.01.003,"Barratt, Ruth; Shaban, Ramon Z.; Gilbert, Gwen...","Infection, Disease & Health",2019-08-31,48.759396
17,ULK1/2 Restricts the Formation of Inducible SI...,Membraneless organelles (MLOs) are liquid-like...,10.1016/j.isci.2019.08.001,"Saul, Vera Vivian; Seibert, Markus; Krüger, Ma...",iScience,2019 Aug 6,48.730643


### Creating an Autocomplete Search bar with ranking by score
Here we provide a search bar with autocomplete. This uses IPywidgets interactive rendering of a TextBox.

In [0]:
def search_papers(SearchTerms: str):
    results_to_consider=200
    results_to_display=10
    # gather search results by score
    output = bm25_index.search(SearchTerms, n=results_to_consider)
    # sort results by recency
    # output=search_results.results.sort_values(by=['publish_time'], ascending=False).head(results_to_display)
    if len(output) > 0:
        display(output) 
    return output

searchbar = widgets.interactive(search_papers, SearchTerms='ACE spike')
searchbar

interactive(children=(Text(value='ACE spike', description='SearchTerms'), Output()), _dom_classes=('widget-int…

### TODO

In [0]:
# Do search with option to restrict years available

### Looking at the Covid Research Tasks
This dataset has a number of tasks. We will try to organize the papers according to the tasks

What is known about transmission, incubation, and environmental stability?
What do we know about COVID-19 risk factors?
What do we know about virus genetics, origin, and evolution?
What has been published about ethical and social science considerations?
What do we know about diagnostics and surveillance?
What has been published about medical care?
What do we know about non-pharmaceutical interventions?
What has been published about information sharing and inter-sectoral collaboration?
What do we know about vaccines and therapeutics?

In [0]:
tasks = [('What is known about transmission, incubation, and environmental stability?', 
        'transmission incubation environment coronavirus'),
        ('What do we know about COVID-19 risk factors?', 'risk factors'),
        ('What do we know about virus genetics, origin, and evolution?', 'genetics origin evolution'),
        ('What has been published about ethical and social science considerations','ethics ethical social'),
        ('What do we know about diagnostics and surveillance?','diagnose diagnostic surveillance'),
        ('What has been published about medical care?', 'medical care'),
        ('What do we know about vaccines and therapeutics?', 'vaccines vaccine vaccinate therapeutic therapeutics')] 
tasks = pd.DataFrame(tasks, columns=['Task', 'Keywords'])

#### Research papers for each task
Here we add a dropdown that allows for selection of tasks and show the search results

In [0]:
def show_task(Task):
    print(Task)
    keywords = tasks[tasks.Task == Task].Keywords.values[0]
    search_results = bm25_index.search(keywords, n=200)
    return search_results
    
results = interact(show_task, Task = tasks.Task.tolist());

interactive(children=(Dropdown(description='Task', options=('What is known about transmission, incubation, and…

# Create a BERT sentance encoding search engine 
From: https://towardsdatascience.com/building-a-search-engine-with-bert-and-tensorflow-c6fdc0186c8a
By: Denis Antyukhov
In this experiment, we will use a pre-trained BERT model checkpoint to build a general-purpose text feature extractor.

These things are sometimes referred to as Natural Language Understanding (NLU) modules, because the features they extract are relevant for a wide array of downstream NLP tasks.

One use for these features is in instance-based learning, which relies on computing the similarity of the query to the training samples.

We will illustrate this by building a simple Information Retrieval system using the BERT NLU module for feature extraction.

**The plan for this experiment is:**
1. getting the pre-trained BERT model checkpoint
2. extracting a sub-graph optimized for inference
3. creating a feature extractor with tf.Estimator
4. exploring vector space with T-SNE and Embedding Projector
5. implementing an Information Retrieval engine
6. accelerating search queries with math
7. building a covid research article recommendation system

### Step 1: getting the pre-trained model
We start with a pre-trained english BERT-base model checkpoint.

For configuring and optimizing the graph for inference we will use bert-as-a-service repository, which allows for serving BERT models for remote clients over TCP.

Having a remote BERT-server is beneficial in multi-host environments. However, in this part of the experiment we will focus on creating a local (in-process) feature extractor. This is useful if one wishes to avoid additional latency and potential failure modes introduced by a client-server architecture. Now, let us download the model and install the package.

Now, let us download the model and install the package.

In [0]:
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
!unzip uncased_L-12_H-768_A-12.zip
!pip install bert-serving-server --no-deps

## Step 2: optimizing the inference graph
Normally, to modify the model graph we would have to do some low-level TensorFlow programming. 

However, thanks to bert-as-a-service, we can configure the inference graph using a simple CLI interface.

There are a couple of parameters in the below snippet too look out for.

For each text sample, BERT-base model encoding layers output a tensor of shape **[sequence_len, encoder_dim],** with one vector per input token. To obtain a fixed representation, we need to apply some sort of pooling.

**POOL_STRAT** parameter defines the pooling strategy applied to the  **POOL_LAYER** encoding layer. The default value **REDUCE_MEAN** averages the vectors for all tokens in a sequence. This strategy works best for most sentence-level tasks, when the model is not fine-tuned. Another option is NONE, in which case no pooling is applied at all. This is useful for word-level tasks such as Named Entity Recognition or POS tagging. For a detailed discussion of other options check out the Han Xiao's [blog post.](https://hanxiao.github.io/2019/01/02/Serving-Google-BERT-in-Production-using-Tensorflow-and-ZeroMQ/)

**SEQ_LEN** affects the maximum length of sequences processed by the model. Smaller values increase the model inference speed almost linearly.

In [0]:
import os
import tensorflow as tf
sesh = tf.InteractiveSession()

from bert_serving.server.graph import optimize_graph
from bert_serving.server.helper import get_args_parser

# input dir
MODEL_DIR = '/content/uncased_L-12_H-768_A-12' #@param {type:"string"}
# output dir
GRAPH_DIR = '/content/graph/' #@param {type:"string"}
# output filename
GRAPH_OUT = 'extractor.pbtxt' #@param {type:"string"}

POOL_STRAT = 'REDUCE_MEAN' #@param ['REDUCE_MEAN', 'REDUCE_MAX', "NONE"]
POOL_LAYER = '-2' #@param {type:"string"}
SEQ_LEN = '256' #@param {type:"string"}

tf.gfile.MkDir(GRAPH_DIR)

parser = get_args_parser()
carg = parser.parse_args(args=['-model_dir', MODEL_DIR,
                               '-graph_tmp_dir', GRAPH_DIR,
                               '-max_seq_len', str(SEQ_LEN),
                               '-pooling_layer', str(POOL_LAYER),
                               '-pooling_strategy', POOL_STRAT])

tmp_name, config = optimize_graph(carg)
graph_fout = os.path.join(GRAPH_DIR, GRAPH_OUT)

tf.gfile.Rename(
    tmp_name,
    graph_fout,
    overwrite=True
)
print("\nSerialized graph to {}".format(graph_fout))

I:GRAPHOPT:[gra:opt: 53]:model config: /content/uncased_L-12_H-768_A-12/bert_config.json
I:GRAPHOPT:[gra:opt: 56]:checkpoint: /content/uncased_L-12_H-768_A-12/bert_model.ckpt
I:GRAPHOPT:[gra:opt: 60]:build graph...
I:GRAPHOPT:[gra:opt:132]:load parameters from checkpoint...
I:GRAPHOPT:[gra:opt:136]:optimize...
I:GRAPHOPT:[gra:opt:144]:freeze...
I:GRAPHOPT:[gra:opt:149]:write graph to a tmp file: /content/graph/tmpku3r0q6a



Serialized graph to /content/graph/extractor.pbtxt


Running the above snippet will put the BERT model graph and weights from  **MODEL_DIR** into a GraphDef object which will be serialized to a pbtxt file at **GRAPH_OUT**. The file will be smaller than the original model because the nodes and variables required for training will be removed. This results in a quite portable solution: for example the english base model only takes 389 MB after exporting.

### Step 3: creating a feature extractor
Now, we will use the serialized graph to build a feature extractor using the tf.Estimator API. We will need to define two things: **input_fn** and **model_fn**

In [0]:
import logging
import numpy as np

from tensorflow.python.estimator.estimator import Estimator
from tensorflow.python.estimator.run_config import RunConfig
from tensorflow.python.estimator.model_fn import EstimatorSpec
from tensorflow.keras.utils import Progbar

from bert_serving.server.bert.tokenization import FullTokenizer
from bert_serving.server.bert.extract_features import convert_lst_to_features


log = logging.getLogger('tensorflow')
log.setLevel(logging.INFO)
log.handlers = []

In [0]:
GRAPH_PATH = "/content/graph/extractor.pbtxt" #@param {type:"string"}
VOCAB_PATH = "/content/uncased_L-12_H-768_A-12/vocab.txt" #@param {type:"string"}

SEQ_LEN = 256 #@param {type:"integer"}

**input_fn** manages getting the data into the model. That includes executing the whole text preprocessing pipeline and preparing a feed_dict for BERT. 

First, each text sample is converted into a tf.Example instance containing the necessary features listed in **INPUT_NAMES**. The bert_tokenizer object contains  the WordPiece vocabulary and performs the text preprocessing. After that the examples are re-grouped by feature name in a **feed_dict**.

In [0]:
INPUT_NAMES = ['input_ids', 'input_mask', 'input_type_ids']
bert_tokenizer = FullTokenizer(VOCAB_PATH)

def build_feed_dict(texts):
    
    text_features = list(convert_lst_to_features(
        texts, SEQ_LEN, SEQ_LEN, 
        bert_tokenizer, log, False, False))

    target_shape = (len(texts), -1)

    feed_dict = {}
    for iname in INPUT_NAMES:
        features_i = np.array([getattr(f, iname) for f in text_features])
        features_i = features_i.reshape(target_shape).astype("int32")
        feed_dict[iname] = features_i

    return feed_dict

tf.Estimators have a fun feature which makes them re-build and re-initialize the whole computational graph at each call to the predict function. 

So, in order to avoid the overhead, to the predict function we will pass a generator, which will yield the features to the model in a never-ending loop.

In [0]:
def build_input_fn(container):
    
    def gen():
        while True:
          try:
            yield build_feed_dict(container.get())
          except:
            yield build_feed_dict(container.get())

    def input_fn():
        return tf.data.Dataset.from_generator(
            gen,
            output_types={iname: tf.int32 for iname in INPUT_NAMES},
            output_shapes={iname: (None, None) for iname in INPUT_NAMES})
    return input_fn

class DataContainer:
  def __init__(self):
    self._texts = None
  
  def set(self, texts):
    if type(texts) is str:
      texts = [texts]
    self._texts = texts
    
  def get(self):
    return self._texts

**model_fn** contains the specification of the model. In our case, it is loaded from the pbtxt file we saved in the previous step. 

The features are mapped explicitly to the corresponding input nodes with input_map.

In [0]:
def model_fn(features, mode):
    with tf.gfile.GFile(GRAPH_PATH, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        
    output = tf.import_graph_def(graph_def,
                                 input_map={k + ':0': features[k] for k in INPUT_NAMES},
                                 return_elements=['final_encodes:0'])

    return EstimatorSpec(mode=mode, predictions={'output': output[0]})
  
estimator = Estimator(model_fn=model_fn)

Using temporary folder as model directory: /tmp/tmpqgabu32k


Now we have everything we need to perform inference:

In [0]:
def batch(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

def build_vectorizer(_estimator, _input_fn_builder, batch_size=128):
  container = DataContainer()
  predict_fn = _estimator.predict(_input_fn_builder(container), yield_single_examples=False)
  
  def vectorize(text, verbose=False):
    x = []
    bar = Progbar(len(text))
    for text_batch in batch(text, batch_size):
      container.set(text_batch)
      x.append(next(predict_fn)['output'])
      if verbose:
        bar.add(len(text_batch))
      
    r = np.vstack(x)
    return r
  
  return vectorize

In [0]:
bert_vectorizer = build_vectorizer(estimator, build_input_fn)

In [0]:
bert_vectorizer(64*['sample text']).shape

From /tensorflow-1.15.0/python3.6/tensorflow_core/python/ops/array_ops.py:1475: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


(64, 768)

### Step 4: exploring vector space with Projector

*A* standalone version of BERT feature extractor is available in the [repository](https://github.com/gaphex/bert_experimental).

Using the vectorizer we will generate embeddings for articles from the CORD-19 benchmark (in this tutorial, the Reuters-21578 benchmark corpus was used previously)

To visualise and explore the embedding vector space in 3D we will use a dimensionality reduction technique called [T-SNE](https://distill.pub/2016/misread-tsne/).

Lets get the article embeddings first.

In [0]:
import nltk
from nltk.corpus import reuters

nltk.download("reuters")
nltk.download("punkt")

[nltk_data] Downloading package reuters to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
type(reuters)

nltk.corpus.reader.plaintext.CategorizedPlaintextCorpusReader

In [0]:
# REUTERS EXAMPLE
max_samples = 256
categories = ['wheat', 'tea', 'strategic-metal', 
              'housing', 'money-supply', 'fuel']

S, X, Y = [], [], []

for category in categories:
  print(category)
  
  sents = reuters.sents(categories=category)
  sents = [' '.join(sent) for sent in sents][:max_samples]
  X.append(bert_vectorizer(sents, verbose=True))
  Y += [category] * len(sents)
  S += sents
  
X = np.vstack(X) 
X.shape

wheat
256/256 [==============================] - 5s 20ms/step
tea
154/154 [==============================] - 3s 20ms/step
strategic-metal
200/200 [==============================] - 4s 19ms/step
housing
139/139 [==============================] - 3s 20ms/step
money-supply
256/256 [==============================] - 5s 20ms/step
fuel
129/129 [==============================] - 3s 20ms/step


(1134, 768)

In [0]:
with open("embeddings.tsv", "w") as fo:
  for x in X.astype('float16'):
    line = "\t".join([str(v) for v in x])
    fo.write(line + "\n")

with open("metadata.tsv", "w") as fo:
  fo.write("Label\tSentence\n")
  for y, s in zip(Y, S):
    fo.write("{}\t{}\n".format(y, s))

The interactive visualization of generated embeddings is available on the [Embedding Projector](https://projector.tensorflow.org/?config=https://gist.githubusercontent.com/gaphex/7262af1e151957b1e7c638f4922dfe57/raw/3b946229fc58cbefbca2a642502cf51d4f8e81c5/reuters_proj_config.json). **<--CLICK THAT TO GENERATE**

From the link you can run T-SNE yourself, or load a checkpoint using the bookmark in lower-right corner (loading works only on Chrome).

To reproduce the input files used for this visualization, run the code below. Then, download the files to your machine and upload to Projector

(you can dowload files from the menu opened by the ">" button in the upper-left)

In [0]:
from IPython.display import HTML

HTML("""
<video width="900" height="632" controls>
  <source src="https://storage.googleapis.com/bert_resourses/reuters_tsne_hd.mp4" type="video/mp4">
</video>
""")

### Create embeddings for CORD19 Articles

In [0]:
# Convert pandas dataframe to nltk.corpus.reader.plaintext.CategorizedPlaintextCorpusReader
# From: https://stackoverflow.com/questions/49088978/how-to-create-corpus-from-pandas-data-frame-to-operate-with-nltk/49104725
def CreateCorpusFromDataFrame(corpusfolder,df):
    for index, r in df.iterrows():
        id=index
        title=r['title']
        body=r['title']
        # handler text for not properly munged data
        try: 
          category=re.sub('/', '', r['journal']) # remove odd characters as writing to file
        except TypeError:
          continue
        fname=str(category)+'_'+str(id)+'.txt'
        corpusfile=open(corpusfolder+'/'+fname,'a+')
        corpusfile.write(str(body) +" " +str(title))
        corpusfile.close()

In [293]:
# create folder to hold CORD19 nltk
dirName = 'CORD19_nltk_title_only'
try:
    # Create target Directory
    os.mkdir(dirName)
except FileExistsError:
    pass

# create corpus
CreateCorpusFromDataFrame(dirName,metadata)
print("Corpus created in folder: %s" % dirName)

Corpus created in folder: CORD19_nltk_title_only


In [0]:
# Import the corpus reader
from nltk.corpus.reader import CategorizedPlaintextCorpusReader

# Create NLTK data structure (with pattern matching to create the article names again)
CORD_corpus=CategorizedPlaintextCorpusReader(dirName,r'.*', cat_pattern=r'(.*)_.*.txt$') 

In [295]:
# total journals
print("Total number journals: %s" % (len(metadata.journal.unique())))

# select a subset of journals, where the journal will be the tag
num_journals=8
categories=metadata['journal'].value_counts()[:num_journals].index.tolist()
print ("\nPicking most common journals:")
categories



Total number journals: 3929

Picking most common journals:


['PLoS One',
 'Virology',
 'Emerg Infect Dis',
 'Viruses',
 'The Lancet',
 'Sci Rep',
 'Virus Research',
 'Vaccine']

In [296]:
#CORD19 Examples
max_samples = 5000

S, X, Y = [], [], []

for category in categories:
  print(category)
  
  sents = CORD_corpus.sents(categories=category)
  sents = [' '.join(sent) for sent in sents][:max_samples]
  X.append(bert_vectorizer(sents, verbose=True))
  Y += [category] * len(sents)
  S += sents
  
X = np.vstack(X) 
X.shape

PLoS One
1645/1645 [==============================] - 34s 20ms/step
Virology
856/856 [==============================] - 18s 20ms/step
Emerg Infect Dis
811/811 [==============================] - 16s 20ms/step
Viruses
604/604 [==============================] - 12s 20ms/step
The Lancet
741/741 [==============================] - 15s 20ms/step
Sci Rep
493/493 [==============================] - 10s 21ms/step
Virus Research
513/513 [==============================] - 11s 21ms/step
Vaccine
534/534 [==============================] - 11s 21ms/step


(6197, 768)

In [0]:
# make folder in google drive to download files
location = '/content/drive/My Drive/'

with open(location + "embeddings_large.tsv", "w") as fo:
  for x in X.astype('float16'):
    line = "\t".join([str(v) for v in x])
    fo.write(line + "\n")

with open(location + "metadata_large.tsv", "w") as fo:
  fo.write("Label\tSentence\n")
  for y, s in zip(Y, S):
    fo.write("{}\t{}\n".format(y, s))

The interactive visualization of generated embeddings is available on the [Embedding Projector](https://projector.tensorflow.org/?config=https://gist.githubusercontent.com/gaphex/7262af1e151957b1e7c638f4922dfe57/raw/3b946229fc58cbefbca2a642502cf51d4f8e81c5/reuters_proj_config.json). **<--CLICK THAT TO GENERATE**

Then go to bottom right and load in those files